In [1]:
import sys
sys.path.append('/usr/local/lib/python3.5/site-packages')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import os

In [40]:
carpeta = (os.getcwd()+'/tc02Data/')

# Lista de proteinas esenciales
data_ess = pd.read_csv(carpeta+'Essential_ORFs_paperHe.txt', sep='\t', header=0,skipfooter=4,usecols=[1])
data_ess['ORF_name'] = data_ess['ORF_name'].map(lambda x: x.strip()) 
ess = data_ess["ORF_name"].tolist()
del ess[0]

# Lista de grafos
archivos = ['Y2H','AP-MS','LIT','LIT_Reguly']
Gs = []

for j,arc in enumerate(archivos):
    data = pd.read_csv(carpeta+'yeast_'+arc+'.txt', sep='\t', header=None)
    
    G = nx.Graph()
    
    for i in range(len(data)):
        G.add_edges_from([(data[0][i],data[1][i])])
    Gs.append(G)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


In [43]:
# Agrego la caracteristica de ser o no esencial

nodos_G = set(G.nodes()) # set de nodos de G
nodos_ess_G = nodos_G.intersection(set(ess)) # nodos esenciales de G (como interseccion entre nodos de G y esenciales)
nodos_no_ess_G = nodos_G.difference(set(ess)) # nodos no esenciales de G (como diferencia entre nodos de G y esenciales)

# Agrego el atributo correspondiente a cada nodo
G.add_nodes_from(nodos_ess_G, essential=True)
G.add_nodes_from(nodos_no_ess_G, essential=False)

# Para comprobarlo me fije que la cantidad de nodos sea la misma antes y despues de esto


In [15]:
grados_dict = G.degree()

k_lista = list(grados_dict.values())
k_lista = np.unique(k_lista) # vector de grado de nodos

L = len(k_lista)
nodos_ess = np.zeros(L)
nodos_total = np.zeros(L)
nodos_frac = np.zeros(L)

for i,grado in enumerate(k_lista):
    nodos_total[i] = grados_dict.values().count(grado)
    # cuenta cuantas veces aparece cada grado de k_lista

# FALTA CONTAR LOS ESENCIALES